# Tiled with Data from Bluesky

## Connect to a Tiled Service

In [ ]:
from tiled.client import from_uri

In [ ]:
c = from_uri("https://tiled-demo.blueskyproject.io/api")
c

## Navigate to a Catalog of Bluesky Runs

In [ ]:
bmm = c['bmm']['raw']
bmm

Tiled has a mechanism to communicate that this data has a certain "specification" (in this case, a catalog of Bluesky runs) and we use this to display useful information, provide certain conveniences, and achieve speed-ups.

**Aside:** For users who have used Databroker before, compare this to what the "old way" would show:

```python
>>> db
<databroker.v1.Broker at 0x7fb66f1cbd50>  # <-- Not as useful!
```

## Access a specific Bluesky Run

In [ ]:
from databroker import Broker
db = Broker(bmm)
repr(db[22521])

In [ ]:
bmm[22521]  # Access (latest) scan_id 22521.

**Aside:** For users who have used Databroker before, compare this to what the "old way" would show:

```python
>>> db[22521]
<databroker.v1.Header object at 0x7fb66f1e3ed0>  # <-- Not as useful!
```

## Drill in to get data of interest

In [ ]:
bmm[22521]['primary']

In [ ]:
bmm[22521]['primary']['data']

In [ ]:
bmm[22521]['primary']['data'].read()

In [ ]:
bmm[22521]['primary']['data']["I0"]

In [ ]:
bmm[22521]['primary']['data'].read(["dcm_energy", "I0", "It"])

In [ ]:
bmm[22521]['primary']['data'].export("data.csv", variables=["dcm_energy", "I0", "It"])

## Small convenience functions are your friend!

In [ ]:
def load_scan(scan_id):
    return bmm[scan_id]['primary']['data'].read(["I0", "It"])

In [ ]:
load_scan(22521)

## Access metadata

In [ ]:
bmm[22521].metadata  # Spoiler alert: there is a lot!

## Search

In [ ]:
bmm

In [ ]:
from databroker.queries import TimeRange, FullText

bmm.search(TimeRange(since="2020-4-1", until="2021-10-31"))

Does that feel verbose? Again, convenience functions are your friend.

In [ ]:
def Since(since):
    return TimeRange(since=since)

In [ ]:
bmm.search(Since("2020-4-1"))

### Chain searches to progressively narrow results

In [ ]:
bmm.search(Since("2020")).search(FullText("Ni"))

**Aside:** For users who have used Databroker before, compare this to what the "old way" would return:

```python
>>> db("Ni")
<databroker.v1.Results at 0x7fb66e8b8610>  # <-- Not as useful!
```

### Count results

In [ ]:
results = bmm.search(Since("2020"))

len(results)

### Access the first result

In [ ]:
results.values_indexer[0]

### Access the first N results

In [ ]:
results.values_indexer[:5]

### Loop over search results

In [ ]:
for result in results.values():
    # Do something useful here
    print(result)
    ...
    break  # Remove this line to actually run the full loop.

## Access large array (e.g. image time series) data

There are a couple ways to do this. One is to use "Dask" to defer I/O (and, if you like, computation).

In [ ]:
c = from_uri("https://tiled-demo.blueskyproject.io/api", "dask")

In [ ]:
c["fxi"]["raw"][62611]["primary"]["data"]["Andor_image"]

In [ ]:
da = c["fxi"]["raw"][62611]["primary"]["data"]["Andor_image"][22, 10]
da

In [ ]:
img = da.load()

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
plt.imshow(img)